# How can restaurant hook potential Yelp reviews with high-stars
- Data Description: https://www.yelp.com/dataset/documentation/main

In [1]:
import json
import numpy as np
import pandas as pd
import datetime as dt
from afinn import Afinn
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

afinn = Afinn(emoticons=True)
analyzer = SentimentIntensityAnalyzer()

In [2]:
LIMIT = 1000

## 1. Data Loading

### Reviews

- review id: string, 22 character unique review id
- user id: string, 22 character unique user id, maps to the user in user.json
- business id: string, 22 character business id, maps to business in business.json
- stars: integer, star rating
- date: string, date formatted YYYY-MM-DD
- text: string, the review itself
- useful: integer, number of useful votes received
- funny: integer, number of funny votes received
- cool: integer, number of cool votes received

In [3]:
review_dict = {'review_id': [], 'user_id': [], 'business_id': [], 'stars': [], 'useful': [], 'funny': [], 
               'cool': [], 'text': []}
with open('Data/review.json','r') as f:
    for i, line in enumerate(f):
        line_dict = json.loads(line)
        for key in review_dict.keys():
            review_dict[key].append(line_dict[key])
        if i == LIMIT:
            break

review_df = pd.DataFrame(review_dict)
review_df.head()

,business_id,cool,funny,review_id,stars,text,useful,user_id
0,ujmEBvifdJM6h6RLv4wQIg,0,1,Q1sbwvVQXV2734tPgoKj4Q,1.0,Total bill for this horrible service? Over $8G...,6,hG7b0MtEbXx5QzbzE6C_VA
1,NZnhc2sEQy3RmzKTZnqtwQ,0,0,GJXCdrto3ASJOqKeVWPi6Q,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,0,yXQM5uF2jS6es16SJzNHfg
2,WTqjgwHlXbSFevF32_DJVw,0,0,2TzJjDVDEuAW6MR5Vuc1ug,5.0,I have to say that this office really has it t...,3,n6-Gk65cPZL6Uz8qRm3NYw
3,ikCg8xy5JIg_NGPx-MSIDA,0,0,yi0R0Ugj_xUx_Nek0-_Qig,5.0,Went in for a lunch. Steak sandwich was delici...,0,dacAIZ6fTM6mqwW5uxkskg
4,b1b1eb3uo-w561D0ZfCEiQ,0,0,11a8sVPMUFtaC7_ABRkmtw,1.0,Today was my second out of three sessions I ha...,7,ssoyf2_x0EQMed6fgHeMyQ


In [4]:
review_df['text_length'] = review_df['text'].apply(len)
review_df.head()

,business_id,cool,funny,review_id,stars,text,useful,user_id,text_length
0,ujmEBvifdJM6h6RLv4wQIg,0,1,Q1sbwvVQXV2734tPgoKj4Q,1.0,Total bill for this horrible service? Over $8G...,6,hG7b0MtEbXx5QzbzE6C_VA,204
1,NZnhc2sEQy3RmzKTZnqtwQ,0,0,GJXCdrto3ASJOqKeVWPi6Q,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,0,yXQM5uF2jS6es16SJzNHfg,1561
2,WTqjgwHlXbSFevF32_DJVw,0,0,2TzJjDVDEuAW6MR5Vuc1ug,5.0,I have to say that this office really has it t...,3,n6-Gk65cPZL6Uz8qRm3NYw,615
3,ikCg8xy5JIg_NGPx-MSIDA,0,0,yi0R0Ugj_xUx_Nek0-_Qig,5.0,Went in for a lunch. Steak sandwich was delici...,0,dacAIZ6fTM6mqwW5uxkskg,407
4,b1b1eb3uo-w561D0ZfCEiQ,0,0,11a8sVPMUFtaC7_ABRkmtw,1.0,Today was my second out of three sessions I ha...,7,ssoyf2_x0EQMed6fgHeMyQ,3509


In [5]:
def Mean(scores):
    if len(scores) == 0:
        return 0.0
    else:
        return sum(scores) / len(scores)

review_df['afinn_scores'] = review_df['text'].map(afinn.scores)
review_df['afinn_score'] = review_df['afinn_scores'].map(Mean)
review_df.head()

,business_id,cool,funny,review_id,stars,text,useful,user_id,text_length,afinn_scores,afinn_score
0,ujmEBvifdJM6h6RLv4wQIg,0,1,Q1sbwvVQXV2734tPgoKj4Q,1.0,Total bill for this horrible service? Over $8G...,6,hG7b0MtEbXx5QzbzE6C_VA,204,"[-3, -1]",-2.000000
1,NZnhc2sEQy3RmzKTZnqtwQ,0,0,GJXCdrto3ASJOqKeVWPi6Q,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,0,yXQM5uF2jS6es16SJzNHfg,1561,"[3, -1, 3, 3, -1, 2, 1, 3, 3, 2, -3, 2, 3, 3, ...",2.074074
2,WTqjgwHlXbSFevF32_DJVw,0,0,2TzJjDVDEuAW6MR5Vuc1ug,5.0,I have to say that this office really has it t...,3,n6-Gk65cPZL6Uz8qRm3NYw,615,"[2, 3, 2, 4, 1, 2, 2, 2, 3]",2.333333
3,ikCg8xy5JIg_NGPx-MSIDA,0,0,yi0R0Ugj_xUx_Nek0-_Qig,5.0,Went in for a lunch. Steak sandwich was delici...,0,dacAIZ6fTM6mqwW5uxkskg,407,"[3, 3, 3, 3, 3, 1, 3, 2, 3, 3]",2.700000
4,b1b1eb3uo-w561D0ZfCEiQ,0,0,11a8sVPMUFtaC7_ABRkmtw,1.0,Today was my second out of three sessions I ha...,7,ssoyf2_x0EQMed6fgHeMyQ,3509,"[2, 3, -2, -1, -2, 2, -2, -3, -2, 1, -2, -3, -...",-0.941176


In [6]:
def detect_polarity(text):
    return TextBlob(text).sentiment.polarity

review_df['polarity'] = review_df['text'].apply(detect_polarity)
review_df.head()

,business_id,cool,funny,review_id,stars,text,useful,user_id,text_length,afinn_scores,afinn_score,polarity
0,ujmEBvifdJM6h6RLv4wQIg,0,1,Q1sbwvVQXV2734tPgoKj4Q,1.0,Total bill for this horrible service? Over $8G...,6,hG7b0MtEbXx5QzbzE6C_VA,204,"[-3, -1]",-2.000000,-0.333333
1,NZnhc2sEQy3RmzKTZnqtwQ,0,0,GJXCdrto3ASJOqKeVWPi6Q,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,0,yXQM5uF2jS6es16SJzNHfg,1561,"[3, -1, 3, 3, -1, 2, 1, 3, 3, 2, -3, 2, 3, 3, ...",2.074074,0.315810
2,WTqjgwHlXbSFevF32_DJVw,0,0,2TzJjDVDEuAW6MR5Vuc1ug,5.0,I have to say that this office really has it t...,3,n6-Gk65cPZL6Uz8qRm3NYw,615,"[2, 3, 2, 4, 1, 2, 2, 2, 3]",2.333333,0.429125
3,ikCg8xy5JIg_NGPx-MSIDA,0,0,yi0R0Ugj_xUx_Nek0-_Qig,5.0,Went in for a lunch. Steak sandwich was delici...,0,dacAIZ6fTM6mqwW5uxkskg,407,"[3, 3, 3, 3, 3, 1, 3, 2, 3, 3]",2.700000,0.832500
4,b1b1eb3uo-w561D0ZfCEiQ,0,0,11a8sVPMUFtaC7_ABRkmtw,1.0,Today was my second out of three sessions I ha...,7,ssoyf2_x0EQMed6fgHeMyQ,3509,"[2, 3, -2, -1, -2, 2, -2, -3, -2, 1, -2, -3, -...",-0.941176,0.049277


In [7]:
def detect_compound(text):
    return analyzer.polarity_scores(text).get('compound')
def detect_positive(text):
    return analyzer.polarity_scores(text).get('pos')
def detect_negative(text):
    return analyzer.polarity_scores(text).get('neg')
def detect_neutral(text):
    return analyzer.polarity_scores(text).get('neu')

review_df['compound'] = review_df['text'].apply(detect_compound)
review_df['positive'] = review_df['text'].apply(detect_positive)
review_df['negative'] = review_df['text'].apply(detect_negative)
review_df['neutral'] = review_df['text'].apply(detect_neutral)
review_df.head()

,business_id,cool,funny,review_id,stars,text,useful,user_id,text_length,afinn_scores,afinn_score,polarity,compound,positive,negative,neutral
0,ujmEBvifdJM6h6RLv4wQIg,0,1,Q1sbwvVQXV2734tPgoKj4Q,1.0,Total bill for this horrible service? Over $8G...,6,hG7b0MtEbXx5QzbzE6C_VA,204,"[-3, -1]",-2.000000,-0.333333,-0.7661,0.000,0.159,0.841
1,NZnhc2sEQy3RmzKTZnqtwQ,0,0,GJXCdrto3ASJOqKeVWPi6Q,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,0,yXQM5uF2jS6es16SJzNHfg,1561,"[3, -1, 3, 3, -1, 2, 1, 3, 3, 2, -3, 2, 3, 3, ...",2.074074,0.315810,0.9971,0.244,0.026,0.729
2,WTqjgwHlXbSFevF32_DJVw,0,0,2TzJjDVDEuAW6MR5Vuc1ug,5.0,I have to say that this office really has it t...,3,n6-Gk65cPZL6Uz8qRm3NYw,615,"[2, 3, 2, 4, 1, 2, 2, 2, 3]",2.333333,0.429125,0.9810,0.241,0.000,0.759
3,ikCg8xy5JIg_NGPx-MSIDA,0,0,yi0R0Ugj_xUx_Nek0-_Qig,5.0,Went in for a lunch. Steak sandwich was delici...,0,dacAIZ6fTM6mqwW5uxkskg,407,"[3, 3, 3, 3, 3, 1, 3, 2, 3, 3]",2.700000,0.832500,0.9909,0.417,0.000,0.583
4,b1b1eb3uo-w561D0ZfCEiQ,0,0,11a8sVPMUFtaC7_ABRkmtw,1.0,Today was my second out of three sessions I ha...,7,ssoyf2_x0EQMed6fgHeMyQ,3509,"[2, 3, -2, -1, -2, 2, -2, -3, -2, 1, -2, -3, -...",-0.941176,0.049277,-0.9907,0.069,0.106,0.825


### Users

- user_id: string, 22 character unique user id, maps to the user in user.json
- name: string, the user's first name
- review_count: integer, the number of reviews they've written
- yelping_since: string, when the user joined Yelp, formatted like YYYY-MM-DD
- friends: array of strings, an array of the user's friend as user_ids
- useful: integer, number of useful votes sent by the user
- funny: integer, number of funny votes sent by the user
- cool: integer, number of cool votes sent by the user
- fans: integer, number of fans the user has
- elite: array of integers, the years the user was elite
- average_stars: float, average rating of all reviews
- compliment_hot: integer, number of hot compliments received by the user
- compliment_more: integer, number of more compliments received by the user
- compliment_profile: integer, number of profile compliments received by the user
- compliment_cute: integer, number of cute compliments received by the user
- compliment_list: integer, number of list compliments received by the user
- compliment_note: integer, number of note compliments received by the user
- compliment_plain: integer, number of plain compliments received by the user
- compliment_cool: integer, number of cool compliments received by the user
- compliment_funny: integer, number of funny compliments received by the user
- compliment_writer: integer, number of writer compliments received by the user
- compliment_photos: integer, number of photo compliments received by the user

In [8]:
user_dict = {'user_id': [], 'review_count': [], 'yelping_since': [], 'friends': [], 'useful': [], 'funny': [], 
             'cool': [], 'fans': [], 'elite': [], "average_stars": [], "compliment_hot": [], "compliment_more": [], 
             "compliment_profile": [], "compliment_cute": [], "compliment_list": [], "compliment_note": [], 
             "compliment_plain": [], "compliment_cool": [], "compliment_funny": [], "compliment_writer": [], 
             "compliment_photos": []}
with open('Data/user.json','r') as f:
    for i, line in enumerate(f):
        line_dict = json.loads(line)
        for key in user_dict.keys():
            user_dict[key].append(line_dict[key])
        if i == LIMIT:
            break

user_df = pd.DataFrame(user_dict)
user_df.head()

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,compliment_writer,cool,elite,fans,friends,funny,review_count,useful,user_id,yelping_since
0,4.03,1,0,1,2,0,0,1,0,1,...,2,25,"2015,2016,2017",5,"c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g...",17,95,84,l6BmjZMeQD3rDxWUbiAiow,2013-10-08 23:11:33
1,3.63,1,0,1,1,0,0,0,0,0,...,0,16,,4,"kEBTgDvFX754S68FllfCaA, aB2DynOxNOJK9st2ZeGTPg...",22,33,48,4XChL029mKr5hydo79Ljxg,2013-02-21 22:29:06
2,3.71,0,0,0,0,0,0,1,0,0,...,0,10,,0,"4N-HU_T32hLENLntsNKNBg, pSY2vwWLgWfGVAAiKQzMng...",8,16,28,bc8C_eETBWL0olvFSJJd0w,2013-10-04 00:16:10
3,4.85,0,0,0,1,0,0,0,0,2,...,1,14,,5,"RZ6wS38wnlXyj-OOdTzBxA, l5jxZh1KsgI8rMunm-GN6A...",4,17,30,dD0gZpBctWGdWo9WlGuhlA,2014-05-22 15:57:30
4,4.08,80,0,80,28,1,1,16,5,57,...,25,665,"2015,2016,2017,2018",39,"mbwrZ-RS76V1HoJ0bF_Geg, g64lOV39xSLRZO0aQQ6DeQ...",279,361,1114,MM4RJAeH6yuaN8oZDSt0RA,2013-10-23 07:02:50


In [9]:
# calculate the number of elite years
num_elite = []
for i in range(len(user_df)):
    if user_df.elite[i] == '':
        num_elite.append(0)
    else:
        num_elite.append(len(user_df.elite[i].split(',')))
user_df['num_elite'] = num_elite

In [10]:
# calculate friends number
num_friends = []
for i in range(len(user_df)):
    if user_df.friends[i] == '':
        num_friends.append(0)
    else:
        num_friends.append(len(user_df.friends[i].split(',')))
user_df['num_friends'] = num_friends

In [11]:
# calculate how this user have been with yelp
days_with_yelp = []

dates_since = user_df.yelping_since
latest_dates = max(dates_since)

for i in range(len(user_df)):
    days_with_yelp.append((dt.datetime.strptime(latest_dates, "%Y-%m-%d %H:%M:%S") - 
                           dt.datetime.strptime(dates_since[i], "%Y-%m-%d %H:%M:%S")).days)

user_df['days_with_yelp'] = days_with_yelp

In [12]:
user_df.head()

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,fans,friends,funny,review_count,useful,user_id,yelping_since,num_elite,num_friends,days_with_yelp
0,4.03,1,0,1,2,0,0,1,0,1,...,5,"c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g...",17,95,84,l6BmjZMeQD3rDxWUbiAiow,2013-10-08 23:11:33,3,99,1367
1,3.63,1,0,1,1,0,0,0,0,0,...,4,"kEBTgDvFX754S68FllfCaA, aB2DynOxNOJK9st2ZeGTPg...",22,33,48,4XChL029mKr5hydo79Ljxg,2013-02-21 22:29:06,0,1152,1596
2,3.71,0,0,0,0,0,0,1,0,0,...,0,"4N-HU_T32hLENLntsNKNBg, pSY2vwWLgWfGVAAiKQzMng...",8,16,28,bc8C_eETBWL0olvFSJJd0w,2013-10-04 00:16:10,0,15,1372
3,4.85,0,0,0,1,0,0,0,0,2,...,5,"RZ6wS38wnlXyj-OOdTzBxA, l5jxZh1KsgI8rMunm-GN6A...",4,17,30,dD0gZpBctWGdWo9WlGuhlA,2014-05-22 15:57:30,0,525,1142
4,4.08,80,0,80,28,1,1,16,5,57,...,39,"mbwrZ-RS76V1HoJ0bF_Geg, g64lOV39xSLRZO0aQQ6DeQ...",279,361,1114,MM4RJAeH6yuaN8oZDSt0RA,2013-10-23 07:02:50,4,231,1353


### Checkins

- business_id: string, 22 character business id, maps to business in business.json
- date: string which is a comma-separated list of timestamps for each checkin, each with format YYYY-MM-DD HH:MM:SS

In [13]:
checkin_dict = {'business_id': [], 'date': []}
with open('Data/checkin.json','r') as f:
    for i, line in enumerate(f):
        line_dict = json.loads(line)
        for key in checkin_dict.keys():
            checkin_dict[key].append(line_dict[key])
        if i == LIMIT:
            break

checkin_df = pd.DataFrame(checkin_dict)
checkin_df.head()

,business_id,date
0,--1UhMGODdWsrMastO9DZw,"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016..."
1,--6MefnULPED_I942VcFNA,"2011-06-04 18:22:23, 2011-07-23 23:51:33, 2012..."
2,--7zmmkVg-IMGaXbuVd0SQ,"2014-12-29 19:25:50, 2015-01-17 01:49:14, 2015..."
3,--8LPVSo5i0Oo61X01sV9A,2016-07-08 16:43:30
4,--9QQLMTbFzLJ_oT-ON3Xw,"2010-06-26 17:39:07, 2010-08-01 20:06:21, 2010..."


In [14]:
def GapList(times):
    res = []
    for i in range(1, len(times)):
        res.append((dt.datetime.strptime(times[i], "%Y-%m-%d %H:%M:%S") - 
                    dt.datetime.strptime(times[i - 1], "%Y-%m-%d %H:%M:%S")).days)
    if len(res) > 0:
        return (np.mean(res), np.var(res), len(res))
    else:
        return (0.0, 0.0, 0)

checkin_df["avg_checkin"] = checkin_df.date.apply(lambda x: GapList(x.split(", ")))
checkin_df.head()

,business_id,date,avg_checkin
0,--1UhMGODdWsrMastO9DZw,"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016...","(73.6, 3041.4399999999996, 5)"
1,--6MefnULPED_I942VcFNA,"2011-06-04 18:22:23, 2011-07-23 23:51:33, 2012...","(15.789156626506024, 728.0700029031791, 166)"
2,--7zmmkVg-IMGaXbuVd0SQ,"2014-12-29 19:25:50, 2015-01-17 01:49:14, 2015...","(7.5083798882681565, 89.50132642551732, 179)"
3,--8LPVSo5i0Oo61X01sV9A,2016-07-08 16:43:30,"(0.0, 0.0, 0)"
4,--9QQLMTbFzLJ_oT-ON3Xw,"2010-06-26 17:39:07, 2010-08-01 20:06:21, 2010...","(80.41666666666667, 7439.131944444444, 36)"


In [15]:
checkin_df["checkin_count"] = checkin_df["avg_checkin"].apply(lambda x: x[2])
checkin_df["var_checkin"] = checkin_df["avg_checkin"].apply(lambda x: x[1])
checkin_df["avg_checkin"] = checkin_df["avg_checkin"].apply(lambda x: x[0])
checkin_df.head()

,business_id,date,avg_checkin,checkin_count,var_checkin
0,--1UhMGODdWsrMastO9DZw,"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016...",73.600000,5,3041.440000
1,--6MefnULPED_I942VcFNA,"2011-06-04 18:22:23, 2011-07-23 23:51:33, 2012...",15.789157,166,728.070003
2,--7zmmkVg-IMGaXbuVd0SQ,"2014-12-29 19:25:50, 2015-01-17 01:49:14, 2015...",7.508380,179,89.501326
3,--8LPVSo5i0Oo61X01sV9A,2016-07-08 16:43:30,0.000000,0,0.000000
4,--9QQLMTbFzLJ_oT-ON3Xw,"2010-06-26 17:39:07, 2010-08-01 20:06:21, 2010...",80.416667,36,7439.131944


### Tips

- text: string, text of the tip
- date: string, when the tip was written, formatted like YYYY-MM-DD
- compliment_count: integer, how many compliments it has
- business_id: string, 22 character business id, maps to business in business.json
- user_id: string, 22 character unique user id, maps to the user in user.json

In [16]:
tip_dict = {'text': [], 'compliment_count': [], 'business_id': [], 'user_id': []}
with open('Data/tip.json','r') as f:
    for i, line in enumerate(f):
        line_dict = json.loads(line)
        for key in tip_dict.keys():
            tip_dict[key].append(line_dict[key])
        if i == LIMIT:
            break

tip_df = pd.DataFrame(tip_dict)
tip_df.head()

,business_id,compliment_count,text,user_id
0,VaKXUpmWTTWDKbpJ3aQdMw,0,"Great for watching games, ufc, and whatever el...",UPw5DWs_b-e2JRBS-t37Ag
1,OPiPeoJiv92rENwbq76orA,0,Happy Hour 2-4 daily with 1/2 price drinks and...,Ocha4kZBHb4JK0lOWvE0sg
2,5KheTjYPu1HcQzQFtm4_vw,0,Good chips and salsa. Loud at times. Good serv...,jRyO2V1pA4CdVVqCIOPc1Q
3,TkoyGi8J7YFjA6SbaRzrxg,0,The setting and decoration here is amazing. Co...,FuTJWFYm4UKqewaosss1KA
4,AkL6Ous6A1atZejfZXn1Bg,0,Molly is definately taking a picture with Sant...,LUlKtaM3nXd-E4N4uOk_fQ


In [17]:
tip_df['text_length'] = tip_df['text'].apply(len)
tip_df.head()

,business_id,compliment_count,text,user_id,text_length
0,VaKXUpmWTTWDKbpJ3aQdMw,0,"Great for watching games, ufc, and whatever el...",UPw5DWs_b-e2JRBS-t37Ag,66
1,OPiPeoJiv92rENwbq76orA,0,Happy Hour 2-4 daily with 1/2 price drinks and...,Ocha4kZBHb4JK0lOWvE0sg,138
2,5KheTjYPu1HcQzQFtm4_vw,0,Good chips and salsa. Loud at times. Good serv...,jRyO2V1pA4CdVVqCIOPc1Q,104
3,TkoyGi8J7YFjA6SbaRzrxg,0,The setting and decoration here is amazing. Co...,FuTJWFYm4UKqewaosss1KA,96
4,AkL6Ous6A1atZejfZXn1Bg,0,Molly is definately taking a picture with Sant...,LUlKtaM3nXd-E4N4uOk_fQ,52


In [18]:
tip_df['afinn_scores'] = tip_df['text'].map(afinn.scores)
tip_df['afinn_score'] = tip_df['afinn_scores'].map(Mean)
tip_df.head()

,business_id,compliment_count,text,user_id,text_length,afinn_scores,afinn_score
0,VaKXUpmWTTWDKbpJ3aQdMw,0,"Great for watching games, ufc, and whatever el...",UPw5DWs_b-e2JRBS-t37Ag,66,[3],3.0
1,OPiPeoJiv92rENwbq76orA,0,Happy Hour 2-4 daily with 1/2 price drinks and...,Ocha4kZBHb4JK0lOWvE0sg,138,[3],3.0
2,5KheTjYPu1HcQzQFtm4_vw,0,Good chips and salsa. Loud at times. Good serv...,jRyO2V1pA4CdVVqCIOPc1Q,104,"[3, 3, -3]",1.0
3,TkoyGi8J7YFjA6SbaRzrxg,0,The setting and decoration here is amazing. Co...,FuTJWFYm4UKqewaosss1KA,96,[4],4.0
4,AkL6Ous6A1atZejfZXn1Bg,0,Molly is definately taking a picture with Sant...,LUlKtaM3nXd-E4N4uOk_fQ,52,[],0.0


In [19]:
tip_df['polarity'] = tip_df['text'].apply(detect_polarity)
tip_df.head()

,business_id,compliment_count,text,user_id,text_length,afinn_scores,afinn_score,polarity
0,VaKXUpmWTTWDKbpJ3aQdMw,0,"Great for watching games, ufc, and whatever el...",UPw5DWs_b-e2JRBS-t37Ag,66,[3],3.0,0.800000
1,OPiPeoJiv92rENwbq76orA,0,Happy Hour 2-4 daily with 1/2 price drinks and...,Ocha4kZBHb4JK0lOWvE0sg,138,[3],3.0,0.158333
2,5KheTjYPu1HcQzQFtm4_vw,0,Good chips and salsa. Loud at times. Good serv...,jRyO2V1pA4CdVVqCIOPc1Q,104,"[3, 3, -3]",1.0,0.125000
3,TkoyGi8J7YFjA6SbaRzrxg,0,The setting and decoration here is amazing. Co...,FuTJWFYm4UKqewaosss1KA,96,[4],4.0,0.300000
4,AkL6Ous6A1atZejfZXn1Bg,0,Molly is definately taking a picture with Sant...,LUlKtaM3nXd-E4N4uOk_fQ,52,[],0.0,0.000000


In [20]:
tip_df['compound'] = tip_df['text'].apply(detect_compound)
tip_df['positive'] = tip_df['text'].apply(detect_positive)
tip_df['negative'] = tip_df['text'].apply(detect_negative)
tip_df['neutral'] = tip_df['text'].apply(detect_neutral)
tip_df.head()

,business_id,compliment_count,text,user_id,text_length,afinn_scores,afinn_score,polarity,compound,positive,negative,neutral
0,VaKXUpmWTTWDKbpJ3aQdMw,0,"Great for watching games, ufc, and whatever el...",UPw5DWs_b-e2JRBS-t37Ag,66,[3],3.0,0.800000,0.6249,0.291,0.000,0.709
1,OPiPeoJiv92rENwbq76orA,0,Happy Hour 2-4 daily with 1/2 price drinks and...,Ocha4kZBHb4JK0lOWvE0sg,138,[3],3.0,0.158333,0.4588,0.140,0.064,0.795
2,5KheTjYPu1HcQzQFtm4_vw,0,Good chips and salsa. Loud at times. Good serv...,jRyO2V1pA4CdVVqCIOPc1Q,104,"[3, 3, -3]",1.0,0.125000,0.2656,0.227,0.146,0.627
3,TkoyGi8J7YFjA6SbaRzrxg,0,The setting and decoration here is amazing. Co...,FuTJWFYm4UKqewaosss1KA,96,[4],4.0,0.300000,0.6239,0.214,0.000,0.786
4,AkL6Ous6A1atZejfZXn1Bg,0,Molly is definately taking a picture with Sant...,LUlKtaM3nXd-E4N4uOk_fQ,52,[],0.0,0.000000,0.0000,0.000,0.000,1.000


### Photos

- photo_id: string, 22 character unique photo id
- business_id: string, 22 character business id, maps to business in business.json
- caption: string, the photo caption, if any
- label: string, the category the photo belongs to, if any

In [21]:
photo_dict = {'business_id': [], 'label': []}
with open('Data/photo.json','r') as f:
    for i, line in enumerate(f):
        line_dict = json.loads(line)
        for key in photo_dict.keys():
            photo_dict[key].append(line_dict[key])
        if i == LIMIT:
            break

photo_df = pd.DataFrame(photo_dict)
photo_df.head()

,business_id,label
0,rcaPajgKOJC2vo_l3xa42A,inside
1,Kn23LDd740SBVJ7mum0fwg,inside
2,ZkGDCVKSdf8m76cnnalL-A,food
3,bF8gv7k_rwZtiDLP2ZB04w,inside
4,50Anorn0DJXFhBr9a9_gHQ,inside


In [22]:
photo_df.groupby('label').count()

,business_id
label,
drink,46
food,271
inside,444
menu,4
outside,236


In [23]:
def List2Dict(labels):
    res = {"inside": 0, "drink": 0, "food": 0, "menu": 0, "outside": 0}
    for label in labels:
        res[label] += 1
    return res

photo_df = photo_df[["business_id", "label"]].groupby("business_id").aggregate(lambda x: List2Dict(list(x)))
photo_df.head()

,label
business_id,
-050d_XIor1NpCuWkbIVaQ,"{'menu': 0, 'outside': 0, 'drink': 0, 'inside'..."
-3zffZUHoY8bQjGfPSoBKQ,"{'menu': 0, 'outside': 0, 'drink': 0, 'inside'..."
-Bdw-5H5C4AYSMGnAvmnzw,"{'menu': 0, 'outside': 0, 'drink': 1, 'inside'..."
-EX_qttyhm5YNb8NWtAcCQ,"{'menu': 0, 'outside': 0, 'drink': 0, 'inside'..."
-JfncbVURtvuzBbof40o1w,"{'menu': 0, 'outside': 0, 'drink': 0, 'inside'..."


In [24]:
photos = {"inside": [], "drink": [], "food": [], "menu": [], "outside": []}

for i in range(len(photo_df)):
    photo = photo_df.iloc[i, 0]
    for key in photo:
        photos[key].append(photo[key])
photos_df = pd.DataFrame(photos)
photos_df.set_index(photo_df.index, inplace=True)
photos_df.head()

,drink,food,inside,menu,outside
business_id,,,,,
-050d_XIor1NpCuWkbIVaQ,0,0,1,0,0
-3zffZUHoY8bQjGfPSoBKQ,0,1,0,0,0
-Bdw-5H5C4AYSMGnAvmnzw,1,0,1,0,0
-EX_qttyhm5YNb8NWtAcCQ,0,0,2,0,0
-JfncbVURtvuzBbof40o1w,0,0,1,0,0


In [25]:
photos_df = photos_df.reset_index()
photos_df.head()

,business_id,drink,food,inside,menu,outside
0,-050d_XIor1NpCuWkbIVaQ,0,0,1,0,0
1,-3zffZUHoY8bQjGfPSoBKQ,0,1,0,0,0
2,-Bdw-5H5C4AYSMGnAvmnzw,1,0,1,0,0
3,-EX_qttyhm5YNb8NWtAcCQ,0,0,2,0,0
4,-JfncbVURtvuzBbof40o1w,0,0,1,0,0


### Business

- business_id: string, 22 character unique string business id
- name: string, the business's name
- address: string, the full address of the business
- city: string, the city
- state: string, 2 character state code, if applicable
- postal code: string, the postal code
- latitude: float, latitude
- longitude: float, longitude
- stars: float, star rating, rounded to half-stars
- review_count: integer, number of reviews
- is_open: integer, 0 or 1 for closed or open, respectively
- attributes: object, business attributes to values. note: some attribute values might be objects
  - "RestaurantsTakeOut": true,
  - "BusinessParking": {"garage": false, "street": true, "validated": false, "lot": false, "valet": false}
- categories: an array of strings of business categories, such as ["Mexican", "Burgers", "Gastropubs"]
- hours: an object of key day to value hours, hours are using a 24hr clock

In [26]:
business_dict = {'business_id': [], 'latitude': [], 'longitude': [], 'stars': [], 'review_count': [], 'is_open': [], 
                 'categories': [], 'hours': []}
with open('Data/business.json','r') as f:
    for i, line in enumerate(f):
        line_dict = json.loads(line)
        for key in business_dict.keys():
            business_dict[key].append(line_dict[key])
        if i == LIMIT:
            break

business_df = pd.DataFrame(business_dict)
business_df.head()

,business_id,categories,hours,is_open,latitude,longitude,review_count,stars
0,1SWheh84yJXfytovILXOAQ,"Golf, Active Life",None,0,33.522143,-112.018481,5,3.0
1,QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...","{'Friday': '9:0-1:0', 'Sunday': '9:0-0:0', 'Tu...",1,43.605499,-79.652289,128,2.5
2,gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese","{'Friday': '17:30-22:0', 'Sunday': '17:30-21:0...",1,35.092564,-80.859132,170,4.0
3,xvX2CttrVhyG2z1dFg_0xw,"Insurance, Financial Services","{'Friday': '8:0-17:0', 'Monday': '8:0-17:0', '...",1,33.455613,-112.395596,3,5.0
4,HhyxOkGAM07SRYtlQ4wMFQ,"Plumbing, Shopping, Local Services, Home Servi...","{'Friday': '7:0-23:0', 'Sunday': '7:0-23:0', '...",1,35.190012,-80.887223,4,4.0


In [27]:
def Split(string):
    if string:
        return len(string.split(", "))
    else:
        return 0

business_df.categories.apply(lambda x: Split(x)).head()

0    2
1    8
2    3
3    2
4    7
Name: categories, dtype: int64

In [28]:
def Split2(string):
    res = {"Monday": None, "Tuesday": None, "Wednesday": None, "Thursday": None, "Friday": None, 
           "Saturday": None, "Sunday": None}
    if string:
        for day in string:
            time = string[day].split("-")
            start = int(time[0].split(":")[0])
            finish = int(time[1].split(":")[0])
            if finish <= start:
                finish += 24
            res[day] = (finish - start)
    return res

business_df.hours.apply(lambda x: Split2(x)).head()

0    {'Friday': None, 'Sunday': None, 'Tuesday': No...
1    {'Friday': 16, 'Sunday': 15, 'Tuesday': 15, 'W...
2    {'Friday': 5, 'Sunday': 4, 'Tuesday': None, 'W...
3    {'Friday': 9, 'Sunday': None, 'Tuesday': 9, 'W...
4    {'Friday': 16, 'Sunday': 16, 'Tuesday': 16, 'W...
Name: hours, dtype: object

In [29]:
business_df["count_categories"] = business_df.categories.apply(lambda x: Split(x))
business_df["open_condition"] = business_df.hours.apply(lambda x: Split2(x))
business_df = business_df[["business_id", "count_categories", "open_condition", "latitude", "longitude", 
                           "review_count", "stars"]]
business_df.head()

,business_id,count_categories,open_condition,latitude,longitude,review_count,stars
0,1SWheh84yJXfytovILXOAQ,2,"{'Friday': None, 'Sunday': None, 'Tuesday': No...",33.522143,-112.018481,5,3.0
1,QXAEGFB4oINsVuTFxEYKFQ,8,"{'Friday': 16, 'Sunday': 15, 'Tuesday': 15, 'W...",43.605499,-79.652289,128,2.5
2,gnKjwL_1w79qoiV3IC_xQQ,3,"{'Friday': 5, 'Sunday': 4, 'Tuesday': None, 'W...",35.092564,-80.859132,170,4.0
3,xvX2CttrVhyG2z1dFg_0xw,2,"{'Friday': 9, 'Sunday': None, 'Tuesday': 9, 'W...",33.455613,-112.395596,3,5.0
4,HhyxOkGAM07SRYtlQ4wMFQ,7,"{'Friday': 16, 'Sunday': 16, 'Tuesday': 16, 'W...",35.190012,-80.887223,4,4.0


In [30]:
for key in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]:
    business_df[key] = business_df.open_condition.apply(lambda x: x[key])
business_df.head()

,business_id,count_categories,open_condition,latitude,longitude,review_count,stars,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,1SWheh84yJXfytovILXOAQ,2,"{'Friday': None, 'Sunday': None, 'Tuesday': No...",33.522143,-112.018481,5,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,QXAEGFB4oINsVuTFxEYKFQ,8,"{'Friday': 16, 'Sunday': 15, 'Tuesday': 15, 'W...",43.605499,-79.652289,128,2.5,15.0,15.0,15.0,15.0,16.0,16.0,15.0
2,gnKjwL_1w79qoiV3IC_xQQ,3,"{'Friday': 5, 'Sunday': 4, 'Tuesday': None, 'W...",35.092564,-80.859132,170,4.0,4.0,NaN,4.0,4.0,5.0,5.0,4.0
3,xvX2CttrVhyG2z1dFg_0xw,2,"{'Friday': 9, 'Sunday': None, 'Tuesday': 9, 'W...",33.455613,-112.395596,3,5.0,9.0,9.0,9.0,9.0,9.0,NaN,NaN
4,HhyxOkGAM07SRYtlQ4wMFQ,7,"{'Friday': 16, 'Sunday': 16, 'Tuesday': 16, 'W...",35.190012,-80.887223,4,4.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0


## 2. Merging

In [31]:
# business_df = pd.read_csv("Data/business.csv", index_col=0)
# checkin_df = pd.read_csv("Data/checkin.csv", index_col=0)
# photos_df = pd.read_csv("Data/photos.csv", index_col=0)
# review_df = pd.read_csv("Data/review.csv", index_col=0)
# user_df = pd.read_csv("Data/user.csv", index_col=0)
# tip_df = pd.read_csv("Data/tip.csv", index_col=0)

In [32]:
review_df.rename(columns={"cool": "review_cool", "funny": "review_funny", "stars": "review_stars", "text": "review", 
                          "useful": "review_useful", "text_length": "review_len", "afinn_score": "review_afinn_score", 
                          "polarity": "review_polarity", "compound": "review_compound", "positive": "review_positive", 
                          "negative": "review_negative", "neutral": "review_neutral"}, inplace=True)
user_df.rename(columns={"average_stars": "user_stars", "cool": "user_cool", "funny": "user_funny", 
                        "useful": "user_useful", "review_count": "user_review_count"}, inplace=True)
tip_df.rename(columns={"compliment_count": "tip_compliment_count", "text_length": "tip_len", 
                       "afinn_score": "tip_afinn_score", "polarity": "tip_polarity", "compound": "tip_compound", 
                       "positive": "tip_positive", "negative": "tip_negative", "neutral": "tip_neutral"}, inplace=True)
business_df.rename(columns={"review_count": "business_review_count", "stars": "business_stars"}, inplace=True)

In [33]:
review_df.drop(columns=["afinn_scores"], inplace=True)
user_df.drop(columns=["elite", "friends", "yelping_since"], inplace=True)
checkin_df.drop(columns=["date"], inplace=True)
tip_df.drop(columns=["text", "afinn_scores"], inplace=True)
business_df.drop(columns=["open_condition"], inplace=True)

In [34]:
business_all = business_df.merge(photos_df, 
                                 how="outer").merge(checkin_df, 
                                                    how="outer").merge(tip_df.
                                                                       groupby("business_id").mean().reset_index(), 
                                                                       how="outer")
business_all.rename(columns={"tip_compliment_count": "b_tip_compliment_count", "tip_len": "b_tip_len", 
                             "tip_afinn_score": "b_tip_afinn_score", "tip_polarity": "b_tip_polarity", 
                             "tip_compound": "b_tip_compound", "tip_positive": "b_tip_positive", 
                             "tip_negative": "b_tip_negative", "tip_neutral": "b_tip_neutral"}, inplace=True)
business_all.fillna(0, inplace=True)
business_all.head()

,business_id,count_categories,latitude,longitude,business_review_count,business_stars,Monday,Tuesday,Wednesday,Thursday,...,checkin_count,var_checkin,b_tip_compliment_count,b_tip_len,b_tip_afinn_score,b_tip_polarity,b_tip_compound,b_tip_positive,b_tip_negative,b_tip_neutral
0,1SWheh84yJXfytovILXOAQ,2.0,33.522143,-112.018481,5.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,QXAEGFB4oINsVuTFxEYKFQ,8.0,43.605499,-79.652289,128.0,2.5,15.0,15.0,15.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,gnKjwL_1w79qoiV3IC_xQQ,3.0,35.092564,-80.859132,170.0,4.0,4.0,0.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,xvX2CttrVhyG2z1dFg_0xw,2.0,33.455613,-112.395596,3.0,5.0,9.0,9.0,9.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,HhyxOkGAM07SRYtlQ4wMFQ,7.0,35.190012,-80.887223,4.0,4.0,16.0,16.0,16.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
Yelp = review_df.merge(tip_df.groupby("user_id").mean().reset_index(), 
                       how="outer").fillna(0).merge(user_df, how="left").merge(business_all, how="left")
Yelp.head()

,business_id,review_cool,review_funny,review_id,review_stars,review,review_useful,user_id,review_len,review_afinn_score,...,checkin_count,var_checkin,b_tip_compliment_count,b_tip_len,b_tip_afinn_score,b_tip_polarity,b_tip_compound,b_tip_positive,b_tip_negative,b_tip_neutral
0,ujmEBvifdJM6h6RLv4wQIg,0.0,1.0,Q1sbwvVQXV2734tPgoKj4Q,1.0,Total bill for this horrible service? Over $8G...,6.0,hG7b0MtEbXx5QzbzE6C_VA,204.0,-2.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NZnhc2sEQy3RmzKTZnqtwQ,0.0,0.0,GJXCdrto3ASJOqKeVWPi6Q,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,0.0,yXQM5uF2jS6es16SJzNHfg,1561.0,2.074074,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WTqjgwHlXbSFevF32_DJVw,0.0,0.0,2TzJjDVDEuAW6MR5Vuc1ug,5.0,I have to say that this office really has it t...,3.0,n6-Gk65cPZL6Uz8qRm3NYw,615.0,2.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ikCg8xy5JIg_NGPx-MSIDA,0.0,0.0,yi0R0Ugj_xUx_Nek0-_Qig,5.0,Went in for a lunch. Steak sandwich was delici...,0.0,dacAIZ6fTM6mqwW5uxkskg,407.0,2.700000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,b1b1eb3uo-w561D0ZfCEiQ,0.0,0.0,11a8sVPMUFtaC7_ABRkmtw,1.0,Today was my second out of three sessions I ha...,7.0,ssoyf2_x0EQMed6fgHeMyQ,3509.0,-0.941176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
